In [1]:
%pip install -q pymupdf4llm aiofiles

Note: you may need to restart the kernel to use updated packages.


In [8]:
import asyncio
import logging
import os
from typing import Dict

import aiofiles
import pymupdf4llm

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

In [9]:
async def office_to_pdf(file_path: str) -> str:
    output_dir = os.path.dirname(file_path)
    base_name = os.path.basename(file_path)
    pdf_file = os.path.join(output_dir, os.path.splitext(base_name)[0] + ".pdf")

    try:
        process = await asyncio.create_subprocess_exec(
            "libreoffice",
            "--headless",
            "--convert-to",
            "pdf",
            file_path,
            "--outdir",
            output_dir,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )
        stdout, stderr = await process.communicate()

        if process.returncode != 0:
            logger.error(
                f"Error converting {file_path} to PDF: {stderr.decode()}"
            )
            raise RuntimeError(f"Failed to convert {file_path} to PDF")

        logger.info(f"Successfully converted {file_path} to PDF")
        return pdf_file
    except Exception as e:
        logger.error(f"Error in office_to_pdf: {str(e)}")
        raise


async def read_markdown_file(file_path: str) -> str:
    try:
        async with aiofiles.open(file_path, mode="r") as file:
            content = await file.read()
        return content
    except Exception as e:
        logger.error(f"Error reading markdown file {file_path}: {str(e)}")
        raise


async def convert_to_markdown(file_path: str) -> Dict[str, str]:
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    metadata = {
        "original_file": file_path,
        "file_type": ext[1:],  # Убираем точку из расширения
        "conversion_method": "",
    }

    converters = {
        (".doc", ".docx", ".rtf"): lambda f: (
            pymupdf4llm.to_markdown(
                f,
                write_images=False,
                embed_images=False,
                graphics_limit=None,
                margins=(0, 0, 0, 0),
                table_strategy="lines_strict",
                fontsize_limit=1,
                ignore_code=True,
                show_progress=False,
            ),
            "office_to_pdf",
        ),
        ".pdf": lambda f: (
            pymupdf4llm.to_markdown(
                f,
                write_images=False,
                embed_images=False,
                graphics_limit=None,
                margins=(0, 0, 0, 0),
                table_strategy="lines_strict",
                fontsize_limit=1,
                ignore_code=True,
                show_progress=False,
            ),
            "direct_pdf",
        ),
        ".md": read_markdown_file,
    }

    try:
        if ext in (".doc", ".docx", ".rtf"):
            pdf_file = await office_to_pdf(file_path)
            content, method = converters[".pdf"](pdf_file)
        elif ext == ".md":
            content = await converters[".md"](file_path)
            method = "direct_markdown"
        elif ext == ".pdf":
            content, method = converters[".pdf"](file_path)
        else:
            raise ValueError(f"Unsupported file extension: {ext}")

        metadata["conversion_method"] = method
        logger.info(f"Successfully converted {file_path} to markdown")
        return {"content": content, "metadata": metadata}
    except Exception as e:
        logger.error(f"Error converting {file_path} to markdown: {str(e)}")
        raise


async def process_file(file_path: str) -> Dict[str, str]:
    try:
        return await convert_to_markdown(file_path)
    except Exception as e:
        logger.error(f"Error processing file {file_path}: {str(e)}")
        raise

In [10]:
# Пример использования
file_path = '/home/michael/Github/nfd-rag-2024/data/public/Б.1 Б1.1 История (Росии).pdf'
markdown_content = await process_file(file_path)

In [6]:
from IPython.display import Markdown

Markdown(markdown_content['content'])

-----

-----

### 1. ПЕРЕЧЕНЬ ПЛАНИРУЕМЫХ РЕЗУЛЬТАТОВ ПО ДИСЦИПЛИНЕ, СООТНЕСЕННЫХ С ПЛАНИРУЕМЫМИ РЕЗУЛЬТАТАМИ ОСВОЕНИЯ ОПОП ВО

 Целью освоения дисциплины «История (России)» является достижение
 следующих результатов образования: ЗНАТЬ:

  предмет, периодизацию, фактический материал;

  комплекс исторических источников по истории России с древнейших времен до начала ХХ в., их типологию;

  основные этапы историографического изучения актуальных проблем отечественной истории;

  современное состояние и уровень разработанности в отечественной и зарубежной историографии отдельных аспектов истории нашей страны с древнейших времен до сегодняшнего дня;

  этапы исторического развития России;

  место и роль России в истории человечества и в современном мире;
 УМЕТЬ:

  анализировать всемирный исторический процесс;

  применять знание отечественной истории;

  использовать в своей профессиональной деятельности
 междисциплинарные подходы;

  применять научные методы анализа данных изучаемых исторических и источников по истории России с древнейших времен до начала XXI в.;
 ВЛАДЕТЬ:

  общепрофессиональными знаниями теории и методов исторических исследований;

  способностью понимать и критически анализировать излагаемую базовую историческую информацию;

  навыками практического использования современных баз данных по истории России до ХХ века и информационно-коммуникационных технологий;

  навыками логического мышления;

  навыками критического восприятия информации.

 ЦЕЛЬ ИЗУЧЕНИЯ ДИСЦИПЛИНЫ:

 - сформировать у студентов комплексное представление о культурно историческом своеобразии России,

 - познакомить с основными закономерностями и особенностями
 исторического процесса, ввести в круг основных проблем современной исторической науки и заинтересовать изучением прошлого своего Отечества. Изучение дисциплины «История (России)», наряду с другими гуманитарными дисциплинами призвано расширить кругозор и повысить общекультурную подготовку специалиста.


-----

### 2. МЕСТО ДИСЦИПЛИНЫ В СТРУКТУРЕ ОПОП ВО

 Учебная дисциплина «История (России)» относится к числу дисциплин
 базовой части программы бакалавриата (Блок 1). Специальные требования к входным знаниям, умениям и навыкам обучающегося не предусматриваются. Преподавание дисциплины основывается на знаниях, умениях и навыках, полученных в процессе изучения предмета «история» на предыдущем уровне образования.
 Данная дисциплина закладывает основу для формирования
 общекультурной компетенции студента ОК-2, ОК-7 в соответствии с ФГОС ВО.
 Дисциплина «История Отечества» изучается на I курсе в I семестре
 образовательной программы бакалавриата.

 РЕЗУЛЬТАТЫ ОБУЧЕНИЯ ДИСЦИПЛИНЕ, ХАРАКТЕРИЗУЮЩИЕ
 ЭТАПЫ ФОРМИРОВАНИЯ КОМПЕТЕНЦИЙ

 Таблица 1

 Результаты освоения
 основной
 образовательной
 Коды программы
 компете (компетенция или Результаты обучения
 нций содержание
 достигнутого уровня
 освоения
 компетенции)
 ОК-2 Способность Знать: анализировать - основные этапы и ключевые события основные этапы и истории России с древности до наших закономерности дней; исторического - выдающихся деятелей отечественной развития общества истории; для формирования - историческую терминологию гражданской позиции - важнейшие достижения культуры и
 системы ценностей, сформировавшиеся в ходе исторического развития
 - основные этапы исторического развития общества, специфику хода исторического процесса;
 - основные этапы и закономерности исторического развития общества;
 - основные этапы и закономерности

1

|Коды компете нций|Результаты освоения основной образовательной программы (компетенция или содержание достигнутого уровня освоения компетенции)|Результаты обучения|
|---|---|---|
|ОК-2|Способность анализировать основные этапы и закономерности исторического развития общества для формирования гражданской позиции|Знать: - основные этапы и ключевые события истории России с древности до наших дней; - выдающихся деятелей отечественной истории; - историческую терминологию - важнейшие достижения культуры и системы ценностей, сформировавшиеся в ходе исторического развития - основные этапы исторического развития общества, специфику хода исторического процесса; - основные этапы и закономерности исторического развития общества; - основные этапы и закономерности|


-----

|Col1|Col2|исторического развития общества, различные подходы к периодизации исторического процесса. Уметь: - выделять основные периоды исторического развития общества, понимать их значение для формирования гражданской позиции; - анализировать основные этапы и закономерности исторического развития общества, аргументировать их значение для формирования гражданской позиции; - самостоятельно извлекать уроки из прошлого опыта, нестандартно подходить к анализу основных периодов исторического процесса, определять значение способности анализа исторического развития общества для формирования гражданской позиции. - осмысливать процессы, события и явления в истории России в их динамике и взаимосвязи, руководствуясь принципами научной объективности и историзма; - извлекать уроки из исторических событий и на их основе принимать осознанные решения Владеть: - методами и приемами формирования гражданской позиции; - четко выраженной собственной гражданской позицией; - собственной активной гражданской позицией, осознанием необходимости транслировать в обществе понимание ее значимости. - навыками устного и письменного изложения своего понимания исторических процессов - навыками участия в дискуссиях и полемике|
|---|---|---|
|ОК-7|способность понимать, критически анализировать и|Знать: - основы базовой исторической информации|


2


-----

|Col1|Col2|Уметь: - применять базовую историческую информацию в научно- исследовательской, образовательной деятельности|
|---|---|---|
|||Владеть: - навыками квалифицированного анализа, комментирования, реферирования и обобщения результатов научных исследований|


### 3. Объем дисциплины в зачетных единицах с указанием академических
 или астрономических часов, выделенных на контактную работу
 обучающихся с преподавателем (по видам учебных занятий)
 и на самостоятельную работу обучающихся

 Всего часов / ЗЕ
 Виды контактной и внеаудиторной
 очная форма заочная форма
 работы
 обучения обучения
 Общий объем дисциплины 144 / 4 144 / 4
 Аудиторная работа 72 8
 в том числе:
 Лекции 22 4
 Лабораторные занятия -  Практические (семинарские) 50 4
 Самостоятельная работа 72 136
 Контрольная работа -  Иные виды работ: -  Курсовое проектирование -  Курсовая работа -  Формы промежуточной аттестации:
 Зачет -  Экзамен 1 сем. 1 сем.

3

|Виды контактной и внеаудиторной работы|Всего часов / ЗЕ|Col3|
|---|---|---|
||очная форма обучения|заочная форма обучения|
|Общий объем дисциплины|144 / 4|144 / 4|
|Аудиторная работа|72|8|
|в том числе:|||
|Лекции|22|4|
|Лабораторные занятия|-|-|
|Практические (семинарские)|50|4|
|Самостоятельная работа|72|136|
|Контрольная работа|-|-|
|Иные виды работ:|-|-|
|Курсовое проектирование|-|-|
|Курсовая работа|-|-|
|Формы промежуточной аттестации:|||
|Зачет|-|-|
|Экзамен|1 сем.|1 сем.|


-----

### 4. СОДЕРЖАНИЕ ДИСЦИПЛИНЫ, СТРУКТУРИРОВАННОЕ
 ПО ТЕМАМ (РАЗДЕЛАМ) С УКАЗАНИЕМ ОТВЕДЕННОГО НА НИХ
 КОЛИЧЕСТВА АКАДЕМИЧЕСКИХ ЧАСОВ И ВИДОВ УЧЕБНЫХ
 ЗАНЯТИЙ
 Таблица 4.1.

 РАЗДЕЛЫ ДИСЦИПЛИНЫ
 В том числе очная (заочная)

 Из них
 Всего
 Наименование
 часов
 №№ разделов Аудиторные Самостоятельная
 очн.
 п/п дисциплины занятия работа
 (заочн.)

 1 История России с
 древнейших 72 36 12 24 36
 времен до начала (72) (4) (2) (2) (68)
 ХХ в.

 2 История России 72 36 10 26 36
 ХХ - ХХI вв. (72) (4) (2) (2) (68)

 Всего часов 144 72 22 50 72
 (144) (8) (4) (4) (136)

 Таблица 4.2.

 ТЕМЫ РАЗДЕЛОВ ДИСЦИПЛИНЫ

**Номер**

**Номер**

**вида**

**Раздел** **заня-** **Вид занятия** **Тема занятия** **Часы**

**заня-**

**тия**

**тий**

### 1 Лекция 1 История в системе социально- 2 гуманитарных наук. Основы
 методологии исторической науки.

 2 Лекция 2 Древняя Русь в IX- п.п. XV вв. 2

 3 Лекция 3 Россия в конце XV-XVII вв.: от 2 великого княжества к царству

 4 Лекция 4 Формирование и развитие Российской 2 империи в XVIII веке

 5 Лекция 5 Российская империя в первой половине 2 XIX века

4

|№№ п/п|Наименование разделов дисциплины|Всего часов очн. (заочн.)|В том числе очная (заочная)|Col5|Col6|Col7|
|---|---|---|---|---|---|---|
||||Аудиторные занятия|Из них||Самостоятельная работа ы|
|||||Лекции|Семинар||
|1|История России с древнейших времен до начала ХХ в.|72 (72)|36 (4)|12 (2)|24 (2)|36 (68)|
|2|История России ХХ - ХХI вв.|72 (72)|36 (4)|10 (2)|26 (2)|36 (68)|
||Всего часов|144 (144)|72 (8)|22 (4)|50 (4)|72 (136)|

|Раздел|Номер заня- тия|Вид занятия|Номер вида заня- тий|Тема занятия|Часы|
|---|---|---|---|---|---|
|Раздел 1. История России с древнейших времен до начала XX в.|1|Лекция|1|История в системе социально- гуманитарных наук. Основы методологии исторической науки.|2|
||2|Лекция|2|Древняя Русь в IX- п.п. XV вв.|2|
||3|Лекция|3|Россия в конце XV-XVII вв.: от великого княжества к царству|2|
||4|Лекция|4|Формирование и развитие Российской империи в XVIII веке|2|
||5|Лекция|5|Российская империя в первой половине XIX века|2|


-----

|Раздел 1. История России с древнейших времен до начала XX в.|6|Лекция|6|Российская империя во второй половине XIX – начале ХХ веков|2|
|---|---|---|---|---|---|
||7|Семинар|1|Введение в предмет. Основные источники и направления историографии истории России|2|
||8|Семинар|2|Русь – первое государство восточных славян|2|
||9|Семинар|3|Древнерусские земли в XIII-XIV вв.|2|
||10|Семинар|4|Формирование единого русского государства в XV веке|2|
||11|Семинар|5|Московское государство при Иване IV Грозном|2|
||12|Семинар|6|Смутное время в России: причины, основные события, последствия|2|
||13|Семинар|7|Московское царство при Алексее Михайловиче|2|
||14|Семинар|8|Формирование Российской империи при Петре I|2|
||15|Семинар|9|«Эпоха дворцовых переворотов» в истории России|2|
||16|Семинар|10|Внешняя политика Российская империи во второй половине XVIII – начале XIX вв.|2|
||17|Семинар|11|Реформы и контрреформы в XIX веке|2|
||18|Семинар|12|Общественная мысль и общественное движение в России в XIX веке|2|
||19|Семинар|13|Расширение границ и этнокультурный облик империи в XIX веке|2|
|Раздел 2. История России в XX-XXI вв.|20|Лекция|7|Россия в годы великих потрясений (1914-1922 гг.)|2|
||21|Лекция|8|Советское государство в 1920-30-е годы|2|
||22|Лекция|9|Великая Отечественная война 1941- 1945 гг.|2|
||23|Лекция|10|Апогей и кризис советской системы 1945-1991 гг.|2|


5


-----

|Раздел 2. История России в XX - XXI вв.|24|Лекция|11|Российская Федерация на современном этапе развития|2|
|---|---|---|---|---|---|
||25|Семинар|14|Россия в годы Первой мировой войны|2|
||26|Семинар|15|Революционные потрясения в России в 1917 году|2|
||27|Семинар|16|Гражданская война в России как общенациональная катастрофа|2|
||28|Семинар|17|Советская модернизация в 1930-е годы: коллективизация сельского хозяйства результаты, цена, издержки.|2|
||29|Семинар|18|Советская модернизация в 1930-е годы: индустриализация: результаты, цена, издержки.||
||30|Семинар|19|СССР накануне и в начальный период Великой Отечественной войны|2|
||31|Семинар|20|Крым в годы Великой Отечественной войны|2|
||32|Семинар|21|Ялтинская конференция 1945 года как опыт формирования международной системы отношений|2|
||33|Семинар|22|Преобразования в СССР в период «оттепели»|2|
||34|Семинар|23|Советское общество в 1970-80-е годы|2|
||35|Семинар|24|Трансформация советского общества в годы «перестройки»||
||36|Семинар|25|Распад СССР: причины и последствия|2|
||ИТОГО аудиторных часов по дисциплине||||72|


### 5. ПЕРЕЧЕНЬ УЧЕБНО-МЕТОДИЧЕСКОГО ОБЕСПЕЧЕНИЯ
 ДЛЯ САМОСТОЯТЕЛЬНОЙ РАБОТЫ ОБУЧАЮЩИХСЯ
 ПО ДИСЦИПЛИНЕ

 1. Истории России: учебник / А.С. Орлов, В.А. Георгиев, Н.Г. Георгиев,
 Т.А. Сивохина. – 4-е 6зд., перераб. И доп. – М.: Проспект, 2016. – 528 с.
 2. История России: учебник и практикум для прикладного бакалавриата /
 под ред. Д.О. Чурарова, С.А. Саркисян. – М.: Издательство Юрайт, 2016. – 431 с.

6


-----

### 3. Орлов А.С., Георгиев В.А., Георгиева Н.Г., Сивохина Т.А. История
 России: Учебник для вузов. М.: Проспект, 2014.
 4. Сахаров А.Н., Боханов А.Н., Шестаков В.А. История России с
 древнейших времен до наших дней. М., 2012. - 768 с.
 5. Федоров В.А. Моряков В.И. Щетинов Ю.А. История России с
 древнейших времен до наших дней. – М.: Изд. КноРус, 2014. – 536 с.

 В качестве самостоятельной работы обучающимся предлагается
 написать творческую работу по одному из хронологических периодов истории России («Записки путешественника») либо по одной из предложенных тем. Студент может предложить свою тему для творческой работы с согласия преподавателя.

 ТЕМАТИКА ТВОРЧЕСКИХ РАБОТ

 1. В.О.Ключевский об истории и историках. 2. Историческая память и музеи (эссе о посещении музея). 3. Выдающиеся люди Таврического университета (мемориальные доски на
 стенах корпусов ТНУ).
 4. Историческая память и проблема переименования улиц, городов в
 современной России.
 5. Проблема исторических памятников: разрушать или устанавливать. 6. Культура Древней Руси: самобытность и заимствования. 7. Восточнославянский мир и Великая Степь. 8. Переписка Ивана Грозного с князем А.Курбским. 9. «День народного единства» (4 ноября): что празднует Россия? 10.Иностранцы о Московском государстве. 11.Переяславская Рада: присоединение или воссоединение? 12.Петр I – революционер на троне? 13.Екатерина II и французские просветители. 14.М.М.Сперанский – светило российской бюрократии. 15.Декабристы и их след в истории. 16.«Народная воля» и террор. 17.Транссибирская магистраль – выдающееся инженерно-техническое
 сооружение.
 18.«Дневник» Николая II – штрихи к портрету последнего императора. 19.Российские революции в названиях улиц и площадей в городах Крыма. 20.Гражданская война в документах. 21.Советский государственный деятель: штрихи к портрету. 22.Нэпманы и совбуры. 23.И.В.Сталин и внутрипартийная борьба в ВКП(б) в 1920-х гг. 24.Политика «коренизации» в Крыму. 25.Советская индустриализация и Крым. 26.Коллективизация в СССР в документах. 27.Великая Отечественная война: «открытия» нацистов о советских людях.

7


-----

### 28.Великая Отечественная война и наука. 29.Великая Отечественная война и ленд-лиз. 30.Великая Отечественная война в воспоминаниях военачальников (на
 примере одного из них).
 31.Великая Отечественная война: героизм советских людей. 32.Создание атомного оружия в СССР. 33.ХХ съезд КПСС: начало «оттепели» в СССР. 34.Диссидентское и правозащитное движение в СССР. 35.Наука и общество: проблемы социальных последствий НТП (на примерах
 открытий и изобретений российских ученых).

 МЕТОДИЧЕСКИЕ РЕКОМЕНДАЦИИ ПО ПОДГОТОВКЕ
 ТВОРЧЕСКОЙ РАБОТЫ «ЗАПИСКИ ПУТЕШЕСТВЕННИКА»

 Данная творческая работа представляет собой небольшой очерк или
 сочинение на заданную тему. По жанру работа близка к эссе, выражает индивидуальные впечатления и соображения по конкретному поводу или вопросу и заведомо не претендует на определяющую или исчерпывающую трактовку предмета.
 Подготовка творческой работы способствует развитию навыков
 самостоятельного творческого мышления и письменного изложения собственных мыслей. Писать эссе чрезвычайно полезно, поскольку это позволяет автору научиться четко и грамотно формулировать мысли, структурировать информацию, использовать основные категории анализа, выделять причинно-следственные связи, иллюстрировать понятия соответствующими примерами, аргументировать свои выводы.
 Цель творческой работы «Записки путешественника» заключается в
 более глубоком изучении и освещении студентом особенностей политического, экономического и культурного развития определенного региона России в указанный период. Перед студентом ставится исследовательская задача – собрать фактический материал, переосмыслить его и выразить свое отношение к историческому прошлому России.
 Исходные данные для написания работы частично предлагаются
 преподавателем, частично выбираются студентом самостоятельно:

  преподавателем распределяются хронологические периоды для каждого студента

  выбор региона путешествия (село, город, княжество, земля, область – в соответствии с административно-территориальным делением русского государства в рамках предложенного хронологического периода) остается за студентом.
 В начале работы указывается (либо прослеживается в работе)
 характеристика героя, от имени которого пишутся «Записки путешественника». Это может быть современный человек, оказавшийся в прошлом, либо современник событий (русского или иностранного

8


-----

### происхождения), либо иное лицо, выбранное автором. При описании впечатлений выбранного героя необходимо придерживаться принципа историзма и рассматривать события и явления в их возникновении и развитии с учетом конкретно-исторической обстановки соответствующей эпохи.
 В содержании творческой работы необходимо уделить внимание
 следующим вопросам:
 1. каковы были особенности политического, экономического и
 культурного развития выбранного региона в указанный хронологический период
 2. свидетелем или участником какого исторического события Вы могли
 оказаться
 3. с кем из исторических персонажей (от известных людей до простого
 обывателя) Вы могли познакомиться
 4. какие впечатления от увиденного и услышанного Вы получили

 Форма изложения материала произвольная, творческий подход
 приветствуется, но с опорой на исторические факты.
 Требования к оформлению работы: титульный лист, объем работы – 5-7
 печатных листов (14 шрифт, 1,5 интервал), список использованных источников и литературы.

 ЛИТЕРАТУРА

 Основная литература:
 1 Истории России: учебник / А.С. Орлов, В.А. Георгиев, Н.Г. Георгиев,
 Т.А. Сивохина. – 4-е 9зд., перераб. И доп. – М.: Проспект, 2016. – 528 с.
 2 История России: учебник и практикум для прикладного бакалавриата /
 под ред. Д.О. Чурарова, С.А. Саркисян. – М.: Издательство Юрайт, 2016. – 431 с.
 3 Орлов А.С., Георгиев В.А., Георгиева Н.Г., Сивохина Т.А. История
 России: Учебник для вузов. М.: Проспект, 2014.
 4 Сахаров А.Н., Боханов А.Н., Шестаков В.А. История России с
 древнейших времен до наших дней. М., 2012. - 768 с.
 5 Федоров В.А. Моряков В.И. Щетинов Ю.А. История России с
 древнейших времен до наших дней. – М.: Изд. КноРус, 2014. – 536 с.

 Дополнительная:
 1. Орлов А.С., Георгиев В.А., Георгиева Н.Г. Хрестоматия по истории
 России: учебное пособие. - М., Проспект. 2014. - 588 с.
 2. Орлов А.С., Георгиев В.А., Георгиева Н.Г., Сивохина Т.А. История
 России в схемах. - М.: Проспект, 2014. - 303 с.

9


-----

### 6. ФОНД ОЦЕНОЧНЫХ СРЕДСТВ ДЛЯ ПРОВЕДЕНИЯ
 ПРОМЕЖУТОЧНОЙ АТТЕСТАЦИИ ОБУЧАЮЩИХСЯ
 ПО ДИСЦИПЛИНЕ

 6.1. Результаты обучения по дисциплине, формы промежуточной аттестации и виды оценочных средств

**Результаты обучения по** **Критерии и шкала оценки** **Формы** **Вид**

**дисциплине, необходимые для** **результатов обучения по** **промеж** **оценочн**

**формирования компетенции** **дисциплине** **уточной** **ых**

**или ее части** **аттеста** **средств**

**ции**

### Знать: Оценки «отлично» А (90- Экзамен Вопросы
 - основные этапы и 100) заслуживает к

обучающийся, экзамену

### ключевые события истории

подтвердивший:

### России с древности до наших

                   - всесторонние,

### дней; систематические и глубокие
 - выдающихся деятелей знания учебного и отечественной истории; нормативного материала по
 - историческую изучаемой дисциплине;

                  - умение свободно выполнять

### терминологию

задания, предусмотренные

### - важнейшие достижения

программой, и

### культуры и системы дополнительной учебной ценностей, литературой, сформировавшиеся в ходе рекомендованной кафедрой; исторического развития - навыки уверенного и

творческого применения

### - основные этапы

полученных знаний и умений

### исторического развития

при решении

### общества, специфику хода профессиональных задач. исторического процесса; Оценки «хорошо» В (82-89)
 - основные этапы и заслуживает обучающийся, закономерности подтвердивший:

                   - полное знание учебного

### исторического развития

материала,

### общества;

предусмотренного

### - основные этапы и программой; закономерности - умение успешно выполнять исторического развития предусмотренные в общества, различные программе задания;

                  - навыки уверенного

### подходы к периодизации

применения полученных

### исторического процесса.

знаний и умений при

### Уметь: решении профессиональных
 - выделять основные задач. периоды исторического Оценки «хорошо» С (74-81) развития общества, понимать заслуживает обучающийся,

подтвердивший:

### их значение для

10

|Результаты обучения по дисциплине, необходимые для формирования компетенции или ее части|Критерии и шкала оценки результатов обучения по дисциплине|Формы промеж уточной аттеста ции|Вид оценочн ых средств|
|---|---|---|---|
|Знать: - основные этапы и ключевые события истории России с древности до наших дней; - выдающихся деятелей отечественной истории; - историческую терминологию - важнейшие достижения культуры и системы ценностей, сформировавшиеся в ходе исторического развития - основные этапы исторического развития общества, специфику хода исторического процесса; - основные этапы и закономерности исторического развития общества; - основные этапы и закономерности исторического развития общества, различные подходы к периодизации исторического процесса. Уметь: - выделять основные периоды исторического развития общества, понимать их значение для|Оценки «отлично» А (90- 100) заслуживает обучающийся, подтвердивший: - всесторонние, систематические и глубокие знания учебного и нормативного материала по изучаемой дисциплине; - умение свободно выполнять задания, предусмотренные программой, и дополнительной учебной литературой, рекомендованной кафедрой; - навыки уверенного и творческого применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» В (82-89) заслуживает обучающийся, подтвердивший: - полное знание учебного материала, предусмотренного программой; - умение успешно выполнять предусмотренные в программе задания; - навыки уверенного применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» С (74-81) заслуживает обучающийся, подтвердивший:|Экзамен|Вопросы к экзамену|


-----

|формирования гражданской позиции; - анализировать основные этапы и закономерности исторического развития общества, аргументировать их значение для формирования гражданской позиции; - самостоятельно извлекать уроки из прошлого опыта, нестандартно подходить к анализу основных периодов исторического процесса, определять значение способности анализа исторического развития общества для формирования гражданской позиции. - осмысливать процессы, события и явления в истории России в их динамике и взаимосвязи, руководствуясь принципами научной объективности и историзма; - извлекать уроки из исторических событий и на их основе принимать осознанные решения Владеть: - методами и приемами формирования гражданской позиции; - четко выраженной собственной гражданской позицией; - собственной активной гражданской позицией, осознанием необходимости транслировать в обществе понимание ее значимости. - навыками устного и письменного изложения своего понимания|- знание учебного материала, предусмотренного программой изучаемой дисциплины; - умение выполнять предусмотренные в программе задания; - навыки применения полученных знаний и умений при решении профессиональных задач. Оценки «удовлетворительно» D (64-73) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в достаточном объеме; - умение справляться с выполнением заданий, предусмотренных программой; - навыки применения полученных знаний и умений при работе с профессиональными задачами. Оценки «удовлетворительно» Е (60-63) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в объеме, необходимом для дальнейшего обучения; - умение частично справляться с выполнением заданий, предусмотренных программой; - навыки фрагментарного применения полученных знаний и умений при работе с профессиональными задачами. Оценки «неудовлетворительно» FX (35-59) заслуживает обучающийся, подтвердивший знания|Col3|Col4|
|---|---|---|---|


11


-----

|исторических процессов - навыками участия в дискуссиях и полемике|основного учебного материала на поверхностном уровне; не справившийся с выполнением заданий, предусмотренных программой. Оценки «неудовлетворительно» F (1-34) заслуживает обучающийся, ответы которого могут быть оценены ниже требований, сформулированных в предыдущем пункте. Необходим повторный курс учебной дисциплины.|Col3|Col4|
|---|---|---|---|
|||||


### ВОПРОСЫ ДЛЯ ПОДГОТОВКИ К ЭКЗАМЕНУ
 1. Предмет и задачи курса «История России». Периодизация истории
 России.
 2. Виды исторических источников. Историография истории России. 3. Теории происхождения человека. Антропогенез. 4. Первобытнообщинный строй: основные черты. 5. Палеолит. Основные черты. Жизнь первых людей в эпоху палеолита. 6. Неолит. Неолитическая революция. 7. Родовая и соседская община: основные черты, причины смены родовой
 общины на соседскую.
 8. Киммерийцы: территория расселения, общественный строй, хозяйство,
 культура.
 9. Скифы: территория расселения, общественный строй, хозяйство,
 культура.
 10.Сарматы: территория расселения, общественный строй, хозяйство,
 культура.
 11.Причины греческой колонизации. Античные города-государства
 Северного Причерноморья. Ольвия. Херсонес. Боспорское царство.
 12.Восточные славяне в древности: территория расселения, общественный
 строй, культура и религиозные верования.
 13.Социально-экономические и политические предпосылки образования
 Древнерусского государства Киевская Русь. Норманнская теория происхождения Руси.
 14.Образование древнерусского государства. Первые киевские князья  Олег, Игорь. Их внутренняя и внешняя политика.
 15.Русь в конце IX - XI вв. Правление княгини Ольги и князя Святослава. 16.Правление князя Владимира. Реформы в древнерусском государстве.
 Христианизация Руси.
 17.Эпоха Ярослава Мудрого: расцвет Руси. Внутренняя и внешняя
 политика князя.

12


-----

### 18.Культура Киевской Руси: художественное ремесло, архитектура,
 искусство, образование, научные знания, летописание.
 19.Распад Руси в XII - начале XIII в. Образование княжеств-государств.
 Причины и последствия политической феодальной раздробленности Руси в XII - начале XIII в.
 20.Монголо-татарская держава. Чингисхан. Вторжение на Русь.
 Восточный поход хана Батыя. Захват Руси. Основные черты монголо- татарского ига.
 21.Владимиро-Суздальское княжество: географическое положение,
 особенности политического устройства, экономика и хозяйство, общественный строй и особенности социального устройства; политическая история.
 22.Новгородская боярская республика: географическое положение,
 особенности политического устройства, экономика и хозяйство, общественный строй и особенности социального устройства; политическая история.
 23.Галицко-Волынское княжество: географическое положение,
 особенности политического устройства, экономика и хозяйство, общественный строй и особенности социального устройства; политическая история.
 24.Северо-Западная Русь: борьба Новгорода и Пскова с экспансией
 крестоносцев на западных границах. Александр Невский.
 25.Юго-Западная Русь: вхождение княжеств в состав Великого княжества
 Литовского.
 26.Северо-Восточная Русь: борьба за великое княжение Владимирское.
 Возвышение Москвы. Дмитрий Донской. Куликовская битва и ее историческое значение.
 27.Формирование единого русского государства в XV веке. Собирание
 земель и укрепление Московского княжества при Иване III Великом. Ликвидация зависимости от Орды.
 28.Избранная рада: время реформ Ивана IV Грозного. 29.Внешняя политика Ивана IV. Взятие Казани и Астрахани. Ливонская
 война. Причины поражения России.
 30.Опричнина Ивана Грозного. 31.Русская культура в XIV - XVI вв. 32.Смутное время в начале XVII века: причины, этапы, последствия. 33.Московское государство при первых Романовых. Система власти и
 эволюция сословного строя.
 34.Внешняя политика России в XVII веке. Вхождение под протекторат
 Московского государства Левобережной Украины - Гетманщины.
 35.Россия в эпоху преобразований Петра I. Особенности петровской
 модернизации.
 36.Эпоха «дворцовых переворотов» после Петра I. 37.Правление Екатерины II и «просвещенный абсолютизм».

13


-----

### 38.Внешняя политика Екатерины II. Борьба России за выход к Черному
 морю. Участие России в разделах Речи Посполитой. Вхождение в состав России Правобережной Украины и Белоруссии.
 39.Культура России в XVIII веке 40.Российская империя в первой половине XIX века. Правление
 Александра I. Проекты либеральных реформ.
 41.Отечественная война 1812 года. 42.Декабристское движение. 43.Император Николай I и его политический режим. Государственный
 консерватизм и его проявления. Крымская война и ее последствия.
 44.Преобразования Александра II. Отмена крепостного права и реформы
 1860-70-х годов в России.
 45.Правление Александра III. Политика контрреформ. Особенности
 внутренней и внешней политики.
 46.Первая российская революция 1905 г.: причины, этапы, ход событий,
 последствия.
 47. Столыпинская аграрная реформа: причины, цели, основные события,
 результат и последствия.
 48.Россия в Первой мировой войне: причины, повод, военные блоки.
 Военные кампании 1914 - 1916 гг. Власть, экономика и общество в условиях войны.
 49.Февральская буржуазно-демократическая революция 1917 г. Отречение
 Николая II и установление власти Временного правительства. Объективные и субъективные причины обострения экономического и политического кризиса.
 50.Октябрьская революция 1917 г. и приход к власти большевиков.
 Первые декреты советской власти. Большевики в 1917 - 1918 гг. Брестский мир.
 51.Гражданская война (1918 - 1922 гг.): причины, этапы и основные
 события Гражданской войны. Военная интервенция.
 52.Политика «военного коммунизма» в годы Гражданской войны (1918  1922 гг.): основные мероприятия, последствия. Террор как главное проявление Гражданской войны.
 53.Советская Россия в 1920-е годы. Переход к новой экономической
 политике: причины, суть, основные преобразования.
 54.Предпосылки, этапы, варианты и значение образования СССР.
 Принятие Конституции СССР 1924 г.
 55.СССР в 1930-е годы. Реализация проекта «сталинского социализма» в
 СССР. Советская модернизация в 1930-е годы: индустриализация: результаты, цена, издержки
 56.СССР в 1930-е годы. Реализация проекта «сталинского социализма» в
 СССР. Советская модернизация в 1930-е годы: коллективизация сельского хозяйства результаты, цена, издержки
 57.Утверждение «культа личности» Сталина. Идеология и политика
 сталинизма. Массовые политические репрессии.

14


-----

### 58.Внешняя политика СССР в 1930-е годы. СССР и международные
 отношения накануне Второй мировой войны (1933 - 1941 гг.)
 59.СССР в начальный период Великой Отечественной войны (июнь1941  осень 1942 гг.): причины неудач, военные операции.
 60.Нацистский оккупационный режим и начало массового сопротивления
 врагу. Партизанское и подпольное движение в годы Великой Отечественной войны.
 61.Коренной перелом в ходе войны (осень 1942 - 1943 гг.).
 Сталинградская битва. Битва на Курской дуге. Завершение освобождения территории СССР.
 62.Внешняя политика СССР в годы войны. Ленд-лиз. Тегеранская
 конференция. Ялтинская и Потсдамская конференции 1945 г. Создание ООН.
 63.Советско-японская война и окончание Второй мировой войны. Итоги
 Великой Отечественной и Второй мировой войны. Источники, уроки и цена Победы.
 64.Особенности послевоенного периода в СССР, восстановление и
 развитие народного хозяйства (1946 - 1953 гг.): ресурсы и приоритеты восстановления.
 65.Послевоенный мир. Рост влияния СССР на международной арене.
 Начало «холодной войны» (1946 - 1953 гг.).
 66.СССР в период «оттепели»: середина 1950-х - первая половина 1960-х
 гг. Н.С. Хрущев и ХХ съезд КПСС. Частичная десталинизация: содержание и противоречия.
 67.Социально-экономическое развитие СССР и преобразования
 Н.С. Хрущева. Новый курс советской внешней политики: от конфронтации к диалогу.
 68.Советское общество в середине 1960-х-начале 1980-х гг.
 Экономические реформы 1960-х гг. Социальная политика: достижения и проблемы. Неосталинизм и концепция «развитого социализма». Конституция 1977 г.
 69.Внешняя политика СССР в 1970-х-начале 1980-х гг.: между разрядкой
 и конфронтацией.
 70.Общественно-политическое развитие СССР и внутренняя политика
 советского руководства в 1964 - 1985 гг. Политика «застоя».
 71.Особенности развития советской культуры в 1950 - 1980-е гг.
 Достижения в науке и технике.
 72.Политика «перестройки» в СССР: сущность, цели, противоречия.
 Международные отношения в 1985 - 1991 гг. Новое политическое мышление и окончание «холодной войны».
 73.Распад СССР: причины, основные события, последствия. Образование
 суверенных государств.
 74.Экономические и политические реформы в России в 1990-е годы.

15


-----

### 75.Россия в начале XXI в. Успехи и противоречия социально экономического и политического развития. Основные направления российской внешней политики.

 ОБРАЗЕЦ ЭКЗАМЕНАЦИОННОГО БИЛЕТА

 1. Предмет и задачи курса «История (России)». Периодизация истории
 России.
 2. Северо-Восточная Русь: борьба за великое княжение Владимирское.
 Возвышение Москвы. Дмитрий Донской. Куликовская битва и ее историческое значение.
 3. Гражданская война (1918 - 1922 гг.): причины, этапы и основные
 события Гражданской войны. Военная интервенция.

 Для проведения срезов знаний обучающихся в период изучения
 дисциплины либо после проведения промежуточной аттестации кафедрой разработаны тестовые задания. Каждый тест направлен на выявление знаний обучающимися событий (дат), терминологии, персоналий, источников и причинно-следственных связей по истории России. Выполнение 5 тестовых заданий 1 теста (выбор правильного ответа) оценивается как высокий уровень, выполнение 4 заданий – достаточный уровень, выполнение 3 заданий – средний уровень, выполнение 1 - 2 заданий – недостаточный уровень.

 ОБРАЗЕЦ ТЕСТА

 1. Первым письменным сводом законов Древней Руси является:
 а) «Судебник»; б) «Повесть временных лет»; в) «Русская Правда»
 2. Назовите неправильную причину возвышения Москвы в XIV веке
 а) отсутствие соперников в борьбе за великокняжеский ярлык; б) перенос митрополичьей кафедры из Владимира в Москву; в) выгодное географическое положение в центре других княжеств.
 3. Кто назывался «временнообязанным» в пореформенной России?
 а) заводчик, бравший на время казенное предприятие в аренду; б) солдат, увольнявшийся после срочной службы в запас; в) бывший крепостной крестьянин, не перешедший на выкуп.
 4. Какое событие произошло в 1919 году:
 а) советско-польская война б) разгром генерала П.Врангеля в) наступление на Москву генерала А.Деникина
 5. С именем какого советского руководителя государства связаны понятия «оттепель» и «десталинизация»?
 а) .И.Брежнев;

16


-----

### б) Н.С.Хрущев; в) М.С.Горбачев.

 6.2. Формы текучего контроля и виды используемых оценочных средств по дисциплине

**Критерии и шкала**

**Разделы** **Результаты**

**оценки результатов** **Вид текущего** **Вид оценочных**

**дисциплин** **изучения разделов**

**изучения разделов** **контроля** **средств**

**ы** **дисциплины**

**дисциплины**

История **ЗНАТЬ:** **Оценки «отлично» А** Выполнение и Задания

России с - предмет, **(90-100) заслуживает** защита творческого

древнейши периодизацию, обучающийся, подтвердивший: индивидуальног уровня,

фактический о творческого позволяющие

х времен до - всесторонние,

материал;

начала ХХ систематические и задания. оценивать и

       - комплекс

глубокие знания диагностировать

века исторических

учебного и умения,

источников по

нормативного интегрировать

истории России до материала по изучаемой

знания различных

начала ХХ в., их дисциплине;

областей,

типологию;        - умение свободно

аргументировать

       - основные этапы выполнять задания,

историографическ предусмотренные собственную
ого изучения программой, и точку зрения.

История актуальных дополнительной Тестирование Задания

учебной литературой, по лекционному репродуктивного

России проблем

рекомендованной материалу. уровня,

ХХ - ХХI отечественной кафедрой;

позволяющие

истории до начала

вв.   - навыки уверенного и

оценивать и

XX в.;

творческого

диагностировать

       - современное применения

знание

состояние и полученных знаний и

фактического

уровень умений при решении
разработанности в профессиональных материала
отечественной и задач. (базовые понятия,
зарубежной **Оценки «хорошо» В** алгоритмы,

**(82-89) заслуживает** факты) и умение

историографии

обучающийся, правильно

отдельных

подтвердивший: использовать

аспектов истории

                   - полное знание

специальные

нашей страны с

учебного материала,

термины и

древнейших

предусмотренного

понятия

времен до ХХ программой;
столетия;        - умение успешно Устное Задания

**УМЕТЬ:** выполнять индивидуальное реконструктивног

       - анализировать предусмотренные в собеседование. о уровня,

всемирный программе задания; позволяющие
исторический        - навыки уверенного оценивать и
процесс; применения диагностировать

       - применять знание полученных знаний и умения

умений при решении

отечественной синтезировать,

профессиональных

истории; анализировать,

задач.

       - использовать в обобщать

**Оценки «хорошо» С**

своей **(74-81) заслуживает** фактический и
профессионально обучающийся, теоретический
й деятельности подтвердивший: материал с
междисциплинарн       - знание учебного формулированием
ые подходы; материала, конкретных

17

|Разделы дисциплин ы|Результаты изучения разделов дисциплины|Критерии и шкала оценки результатов изучения разделов дисциплины|Вид текущего контроля|Вид оценочных средств|
|---|---|---|---|---|
|История России с древнейши х времен до начала ХХ века|ЗНАТЬ: - предмет, периодизацию, фактический материал; - комплекс исторических источников по истории России до начала ХХ в., их типологию; - основные этапы историографическ ого изучения актуальных проблем отечественной истории до начала XX в.; - современное состояние и уровень разработанности в отечественной и зарубежной историографии отдельных аспектов истории нашей страны с древнейших времен до ХХ столетия; УМЕТЬ: - анализировать всемирный исторический процесс; - применять знание отечественной истории; - использовать в своей профессионально й деятельности междисциплинарн ые подходы;|Оценки «отлично» А (90-100) заслуживает обучающийся, подтвердивший: - всесторонние, систематические и глубокие знания учебного и нормативного материала по изучаемой дисциплине; - умение свободно выполнять задания, предусмотренные программой, и дополнительной учебной литературой, рекомендованной кафедрой; - навыки уверенного и творческого применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» В (82-89) заслуживает обучающийся, подтвердивший: - полное знание учебного материала, предусмотренного программой; - умение успешно выполнять предусмотренные в программе задания; - навыки уверенного применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» С (74-81) заслуживает обучающийся, подтвердивший: - знание учебного материала,|Выполнение и защита индивидуальног о творческого задания.|Задания творческого уровня, позволяющие оценивать и диагностировать умения, интегрировать знания различных областей, аргументировать собственную точку зрения.|
|История России ХХ - ХХI вв.|||Тестирование по лекционному материалу.|Задания репродуктивного уровня, позволяющие оценивать и диагностировать знание фактического материала (базовые понятия, алгоритмы, факты) и умение правильно использовать специальные термины и понятия|
||||Устное индивидуальное собеседование.|Задания реконструктивног о уровня, позволяющие оценивать и диагностировать умения синтезировать, анализировать, обобщать фактический и теоретический материал с формулированием конкретных|


-----

|Col1|- применять научные методы анализа данных изучаемых исторических и источников по истории России до начала XX в.; ВЛАДЕТЬ: - общепрофессио- нальными знаниями теории и методов исторических исследований; - способностью понимать и критически анализировать излагаемую базовую историческую информацию.|предусмотренного программой изучаемой дисциплины; - умение выполнять предусмотренные в программе задания; - навыки применения полученных знаний и умений при решении профессиональных задач. Оценки «удовлетворительно» D (64-73) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в достаточном объеме; - умение справляться с выполнением заданий, предусмотренных программой; - навыки применения полученных знаний и умений при работе с профессиональными задачами. Оценки «удовлетворительно» Е (60-63) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в объеме, необходимом для дальнейшего обучения; - умение частично справляться с выполнением заданий, предусмотренных программой; - навыки фрагментарного применения полученных знаний и умений при работе с профессиональными задачами. Оценки «неудовлетворительно » FX (35-59) заслуживает обучающийся, подтвердивший знания основного учебного материала на поверхностном уровне; не справившийся с выполнением заданий, предусмотренных программой. Оценки «неудовлетворительно|Col4|выводов, установлением причинно- следственных связей|
|---|---|---|---|---|


18


-----

|Col1|Col2|» F (1-34) заслуживает обучающийся, ответы которого могут быть оценены ниже требований, сформулированных в предыдущем пункте. Необходим повторный курс учебной дисциплины.|Col4|Col5|
|---|---|---|---|---|


### 6.3. Методические материалы, определяющие процедуры оценивания знаний, умений, навыков и (или) опыта деятельности, характеризующие этапы формирования компетенций

 Общая оценка выставляется с учетом результатов всех этапов
 текущего контроля в порядке, установленном Положением о балльно- рейтинговой системе и Положением о промежуточной аттестации образовательной организации.
 Текущий контроль по дисциплине, является тематическим, и
 процедурно состоит из нескольких этапов:
 1. контроль освоения теоретического материала по каждой теме курса,
 проводится в форме устного опроса (собеседование), тестов (приложение); контрольной работы.
 2. контроль умений применения полученных знаний, проводится в форме
 собеседования по выполнению дополнительных заданий по каждой теме курса, заслушивания и обсуждения стилистических анализов текстов, выполненных обучающимися в рамках тематики дисциплины;
 3. контроль полученных навыков осуществляется в форме выполнения
 практических заданий.
 Текущий контроль проводится в конце разделов курса в форме
 контрольной работы.
 Обучающиеся, не получившие положительную оценку по результатам
 текущего контроля знаний, умений и навыков, формируемых в процессе изучения дисциплины, не допускаются к основному этапу - промежуточному контролю в установленном порядке.
 Промежуточный контроль осуществляется в форме зачета и экзамена,
 которые проводятся в устной форме (собеседование).
 Общая оценка выставляется с учетом результатов всех этапов контроля
 в порядке, установленном кафедрой.

**Виды** **Распределение баллов**

**Формы** **Критерии**

**учебной** **По видам** **Максимальное**

**оценивания** **оценивания**

**деятельности**

**работ** **количество**

Лекция **Письменные** наличие конспекта 0,9 балла 10 баллов

**материалы** в полном объеме

**(конспект)**

19

|Виды учебной деятельности|Формы оценивания|Критерии оценивания|Распределение баллов|Col5|
|---|---|---|---|---|
||||По видам работ|Максимальное количество|
|Лекция|Письменные материалы (конспект)|наличие конспекта в полном объеме|0,9 балла|10 баллов|


-----

|Col1|Col2|каждой прослушанной лекции|Col4|Col5|
|---|---|---|---|---|

|Семинары|Устный опрос (индивидуал ьный и коллективны й в ходе дискуссии)|наличие материалов подготовки к каждому семинару|0,5 балла|40 баллов|
|---|---|---|---|---|
|||развернутый ответ на один из вопросов к 5 семинарским занятиям|1-2 балла||
|||участие в обсуждении темы каждого семинара|0,5 балла||
|||подготовка тематической презентации к одному из семинаров|1-3 балла||
|||подготовка сообщения с дополнительной информацией по теме двух семинаров|1-2 балла||
|Самостояте льная работа|Письменная работа (творческая работа)|умение поставить проблему и выбрать адекватные способы ее решения|1-2 балла|20 баллов|
|||качество и разнообразие использованной исторической информации|1-4 баллов||
|||умение грамотно и ясно изложить содержательную часть|1-6 баллов||
|||наличие собственной точки зрения на проблематику работы|1-3 балла||
|||оригинальность творческих решений в содержании|1-3 балла||
|||оформление работы|1-2 балла||
|Зачет / Экзамен|Устное собеседовани е|знание основных фактов, дат и персоналий|1-5 баллов|30 баллов|
|||владение|1-5 баллов||


20


-----

|Col1|Col2|исторической терминологией|Col4|Col5|
|---|---|---|---|---|
|||умение рассматривать события и явления в историческом контексте, прослеживать причинно- следственные связи|1-10 баллов||
|||умение сформулировать собственную точку зрения по вопросу|1-5 баллов||
|||логичное изложение материала, культура речи|1-5 баллов||
|Всего баллов за один семестр||||100 баллов|


### Процедура оценивания результатов обучения обучающихся основана на
 балльно-рейтинговой системе и состоит в накоплении баллов за разные виды учебной работы в течение семестра. Итоговая оценка выставляется в отчетную документацию согласно ниже представленной таблице.

 Критерии и шкала оценивания

Стобалл Национальная пятибалльная

ьная Шкала шкала оценивания

шкала оценивания Определение

оценива ECTS экзамен зачет

ния

ОТЛИЧНО - отличное

90-100 **А** выполнение с незначительным 5,0 (отлично)

количеством ошибок

ОЧЕНЬ ХОРОШО - выше

82-89 **В** среднего уровня с несколькими

ошибками
4,0 (хорошо)

ХОРОШО - в общем правильная

74-81 **С** работа с определенным зачтено

количеством ошибок

УДОВЛЕТВОРИТЕЛЬНО 
64-73 **D** неплохо, но с большим

количеством недостатков 3,0

ДОСТАТОЧНО - выполнение (достаточно)

60-63 **Е** удовлетворяет минимальным

критериям

35-59 **FX** НЕУДОВЛЕТВОРИТЕЛЬНО - 2,0 не зачтено
возможна повторная сдача (неудовлетворит (с

ельно с возможностью

21

|Стобалл ьная шкала оценива ния|Шкала оценивания ECTS|Определение|Национальная пятибалльная шкала оценивания|Col5|
|---|---|---|---|---|
||||экзамен|зачет|
|90-100|А|ОТЛИЧНО - отличное выполнение с незначительным количеством ошибок|5,0 (отлично)|зачтено|
|82-89|В|ОЧЕНЬ ХОРОШО - выше среднего уровня с несколькими ошибками|4,0 (хорошо)||
|74-81|С|ХОРОШО - в общем правильная работа с определенным количеством ошибок|||
|64-73|D|УДОВЛЕТВОРИТЕЛЬНО - неплохо, но с большим количеством недостатков|3,0 (достаточно)||
|60-63|Е|ДОСТАТОЧНО - выполнение удовлетворяет минимальным критериям|||
|35-59|FX|НЕУДОВЛЕТВОРИТЕЛЬНО - возможна повторная сдача|2,0 (неудовлетворит ельно с|не зачтено (с возможностью|


-----

|Col1|Col2|Col3|возможностью повторной сдачи)|повторной сдачи)|
|---|---|---|---|---|
|1-34|F|НЕУДОВЛЕТВОРИТЕЛЬНО - необходим повторный курс по учебной дисциплине|2,0 (неудовлетворит ельно с повторным изучением дисциплины)|не зачтено (с повторным изучением дисциплины)|


22


-----

### 7. ПЕРЕЧЕНЬ ОСНОВНОЙ И ДОПОЛНИТЕЛЬНОЙ УЧЕБНОЙ
 ЛИТЕРАТУРЫ, НЕОБХОДИМОЙ ДЛЯ ОСВОЕНИЯ ДИСЦИПЛИНЫ

 А. ОСНОВНАЯ УЧЕБНАЯ ЛИТЕРАТУРА:

 1 Истории России: учебник / А.С. Орлов, В.А. Георгиев, Н.Г. Георгиев,
 Т.А. Сивохина. – 4-е 23зд., перераб. И доп. – М.: Проспект, 2016. – 528 с.
 2 История России: учебник и практикум для прикладного бакалавриата /
 под ред. Д.О. Чурарова, С.А. Саркисян. – М.: Издательство Юрайт, 2016. – 431 с.
 3 Орлов А.С., Георгиев В.А., Георгиева Н.Г., Сивохина Т.А. История
 России: Учебник для вузов. М.: Проспект, 2014.
 4 Сахаров А.Н., Боханов А.Н., Шестаков В.А. История России с
 древнейших времен до наших дней. М., 2012. - 768 с.
 5 Федоров В.А. Моряков В.И. Щетинов Ю.А. История России с
 древнейших времен до наших дней. – М.: Изд. КноРус, 2014. – 536 с.

 Б. ДОПОЛНИТЕЛЬНАЯ УЧЕБНАЯ ЛИТЕРАТУРА:

 1. Орлов А.С., Георгиев В.А., Георгиева Н.Г., Сивохина Т.А. История
 России в схемах. - М.: Проспект, 2014. - 303 с.
 2. Орлов А.С., Георгиев В.А., Георгиева Н.Г. Исторический словарь. - М.,
 2014.
 3. Сахаров А.Н. История России с древнейших времен до начала ХХI
 века. - М., 2007. - 1263 с.
 4. Покровский Н.В. Иллюстрированная энциклопедия христианского
 искусства. - М., Эксмо. 2008. - 637 с.

 ПЕРИОДИЧЕСКИЕ ИЗДАНИЯ:
 1. Журнал «Вопросы истории»: официальный сайт  http://rex-history.ru/magazine/233-vi.html
 2. Журнал «История и современность»: официальный сайт  http://www.isras.ru/History&Modernity.html
 3. Журнал «Отечественная история»: официальный сайт  http://annales.info/sbo/contens/oi.htm
 4. Журнал «Русская история»: официальный сайт  http://moscowia.su/projects/zhurnal/zhurnal-russkaya-istoriya-v- bibliotekakh-rossii

23


-----

### 8. ПЕРЕЧЕНЬ РЕСУРСОВ ИНФОРМАЦИОННО ТЕЛЕКОММУНИКАЦИОННОЙ СЕТИ «ИНТЕРНЕТ»,
 НЕОБХОДИМЫХ ДЛЯ ОСВОЕНИЯ ДИСЦИПЛИНЫ

 1. http://histrf.ru/ - История России 2. http://www.runivers.ru/ - Россия в подлиннике 3. http://www.1150russia.ru/ - 1150 лет зарождения российской
 государственности
 4. http://rus-biography.ru/ - Россия: автобиографиии 5. http://militera.lib.ru/ - военная литература 6. Научная электронная библиотека: http://elibrary.ru 7. Электронная библиотека Российской государственной библиотеки
 (РГБ): http://www.rsl.ru/ru/s2/s101/
 8. Хронос. Всемирная история в Интернете: http//www.hrono.ru 9. Виртуальный мир истории. Международный исторический журнал:
 http://history.machaon.ru
 10.Исторические записки. Журнал: http://www.main.vsu.ru 11.Мир истории. Российский электронный журнал: http://www.historia.ru 12.Новая и новейшая история. Журнал: http://www.openweb.ru/nnh/

 8. МЕТОДИЧЕСКИЕ УКАЗАНИЯ ДЛЯ ОБУЧАЮЩИХСЯ
 ПО ОСВОЕНИЮ ДИСЦИПЛИНЫ

 Специфика преподавания курса «История (России)» в высшей школе
 состоит не только в увеличении объема изучаемого материала, но и в особенностях структуры учебной дисциплины. По сравнению со средней школой вводится лекционно-семинарская система. Она предполагает теоретическое изучение предмета на лекциях, самостоятельную подготовку студентов к семинарским занятиям по узловым темам с их последующим обсуждением. Студент обязан посещать все лекции по истории, а следовательно, иметь свои собственные конспекты лекций по предмету. Составление качественного конспекта прослушанных лекций оценивается преподавателем соответствующими баллами.
 Кроме конспектов лекций, обучающийся должен иметь конспекты к
 семинарским занятиям. Семинарские конспекты – это тот материал, который проработан студентом перед практическим занятием. На семинарах заслушиваются и обсуждаются развернутые ответы по вопросам, требующим изучения исторических источников и дополнительной литературы. Для более успешного освоения материала студентам предлагается следующая последовательность подготовки к семинарам.
 Во-первых, внимательно ознакомиться с содержанием плана
 семинарского занятия.

24


-----

### Во-вторых, прочитать конспект лекций и познакомиться с
 соответствующими разделами учебных пособий.
 В-третьих, прочитать рекомендуемую по теме литературу и составить
 конспект прочитанного.
 В-четвертых, провести самоконтроль через соответствующие вопросы. В-пятых, составить план изложения ответа на каждый вопрос
 семинарского занятия. К нескольким семинарам необходимо также подготовить презентацию по одному из вопросов семинара в качестве иллюстративного материала, а также сообщение с дополнительной информацией по теме.
 При оценивании работы студентов на семинарских занятиях
 используется следующая система накопления баллов. Наличие письменных материалов подготовки к семинару и участие в обсуждении темы семинара дает студенту по 1 баллу за каждый семинар (всего 23 балла из 40 баллов). Оставшиеся 17 баллов студент может набрать при подготовке развернутых ответов (минимум 5 ответов) и презентаций (минимум одна) по одному из вопросов семинара, а также за сообщения (минимум два) с дополнительной информацией. 
 Текущая самостоятельная работа студента нацелена на углубление и
 закрепление знаний студента по истории, развитие их практических навыков исторического анализа. К таковой относятся следующие виды деятельности студента:

 - работа с лекционным материалом
 - работа с учебниками и учебными пособиями по дисциплине
 - работа с историческими источниками
 - работа с обязательной и дополнительной литературой, включенной в
 планы семинарских занятий

 - подготовка к экзамену. Творческая проблемно-ориентированная самостоятельная работа
 направлена на развитие интеллектуальных умений, развитие общекультурных компетенций, развитие творческого потенциала личности студента. К такой форме работы относится выполнение студентом творческой работы по истории.
 Изучение курса «История (России)» в учебном плане предусматривает
 промежуточную аттестацию в форме экзамена. Данный вид учебной работы нацелен на выявление достигнутого уровня освоения фактов, закономерностей и особенностей исторического прошлого России. Подготовка к экзамену состоит из двух взаимосвязанных этапов. Первый – систематический труд на протяжении семестра или учебного года, охватывающий все формы учебного процесса: лекции, изучение и конспектирование рекомендованной литературы, активное участие в семинарских занятиях, выполнение самостоятельной работы. Второй –

25


-----

### подготовка непосредственно перед экзаменом, которая позволяет студентам за короткий отрезок времени охватить всю перспективу изученного, систематизировать полученные знания и творчески их переосмыслить. Итоговая оценка по дисциплине учитывает как качество работы студента в течение учебного года, так и проявленный уровень знаний на экзамене.

 10. ПЕРЕЧЕНЬ ИНФОРМАЦИОННЫХ ТЕХНОЛОГИЙ,
 ИСПОЛЬЗУЕМЫХ ПРИ ОСУЩЕСТВЛЕНИИ ОБРАЗОВАТЕЛЬНОГО
 ПРОЦЕССА ПО ДИСЦИПЛИНЕ, ВКЛЮЧАЯ ПЕРЕЧЕНЬ
 ПРОГРАММНОГО ОБЕСПЕЧЕНИЯ И ИНФОРМАЦИОННЫХ
 СПРАВОЧНЫХ СИСТЕМ (ПРИ НЕОБХОДИМОСТИ)

 Для освоения дисциплины «История (России)» обучающийся должен
 иметь представление о программном обеспечении Microsoft Office, в частности Word, Power Point, владеть навыками работы с разными форматами документов (doc, pdf, djvu), пакетом прикладных программ LibreOffice (модули «Презентация», «Текстовый документ» и др.), а также пользоваться поисковыми системами в сети «Интернет».

 11. ОПИСАНИЕ МАТЕРИАЛЬНО-ТЕХНИЧЕСКОЙ БАЗЫ,
 НЕОБХОДИМОЙ ДЛЯ ОСУЩЕСТВЛЕНИЯ ОБРАЗОВАТЕЛЬНОГО
 ПРОЦЕССА ПО ДИСЦИПЛИНЕ

**№** **Наименование оборудованных** **Перечень оборудования и технических**

**п/п** **учебных кабинетов, лабораторий** **средств обучения**

### Лекционная аудитория Мультимедийный проектор, экранное полотно, персональный компьютер (ноутбук) с доступом в Интернет
 Компьютерная аудитория Персональные компьютеры с доступом в Интернет

 Реализация учебной дисциплины требует наличия аудиторий,
 оборудованных проектором, экраном и компьютером. Лекционные занятия проводятся в лекционной аудитории с возможностью использования мультимедийного проектора и выхода в Интернет для демонстрации презентационных материалов, проектированиям на экран видеосюжетов для изучения исторических материалов разных эпох.

26

|№ п/п|Наименование оборудованных учебных кабинетов, лабораторий|Перечень оборудования и технических средств обучения|
|---|---|---|
|1.|Лекционная аудитория|Мультимедийный проектор, экранное полотно, персональный компьютер (ноутбук) с доступом в Интернет|
|2.|Компьютерная аудитория|Персональные компьютеры с доступом в Интернет|


-----

